### Load Dependencies

In [1]:
from utils.HN import *
from utils.model_eval import *
import random

### Define variables
- `class_name2ID` is a dictionary mapping class names to their name. The ID of a species is its position in the Species column of Table 1. It's name is the name given in Table 1. E.g., for the JE-TL19 dataset, : `classID2name = {"Elephant": 0, "Giraffe": 1, "Zebra": 2}`
- The DoR values can be found in Table 12.
- The radii can be found in the Size Avg. column of Table 1. They need to be passed as a dictionary where `key = class ID`, `value = radius`. Again, for the JE-TL19 dataset, the `radii` dict would look as follows: `{0: 62, 1: 81, 2: 49}`


In [ ]:
imgs_dir = "/home/giacomo/ennedi_prepped_640x640_02ovrlp_03neg_ID_2/test_full_imgs"   #TODO: Insert Path to image directory (as downloaded from Zenodo).
model_weights = "/home/giacomo/DELPLANQUE_Results/Ennedi/model_weights/ENNEDI_STEP02_latest_model.pth" #TODO: Insert Path to the model weights you want to run (as downloaded from Zenodo).
class_name2ID = {"Camel": 0, "Donkey": 1, "Sheep/goat": 2}    #TODO: Insert class names and IDs as found in Table 1.
dor_thresh = 0.3    #TODO: Define DoR threshold for NMS and computing evaluation metrics (cf. Table 12)
radii = {0: 80, 1: 50, 2: 25}     #TODO: Insert radius for each species in the dataset; cf. Table 1.
output_dir = "/home/giacomo/projects/How-to-Minimize-the-Annotation-Cost-of-Aerial-Wildlife-Censuses/outputs/AD-ENCR23/HN"     #TODO: Insert path to directory you want your output stored in.

### Run inference
This cell will run HerdNet on the images from the “imgs_dir” entered above and will automatically create a folder inside it (HerdNet_results) in which the .csv file containing the detections will be saved.
- `dets_file`: This is the path to the .csv file containing the HerdNet predictions. One row represents one detection and is expected to have the following columns:
    - `images`: Contains the file names of the images.
    - `x`: Contains the x-coordinate of the detection.
    - `y`: Contains the y-coordinate of the detection.
    - `scores`: Contains the confidence score of the prediction.
    - `species`: Contains the name of the detected species.

    ADD POINT ABOUT PATCH SIZE and OVERLAP
    ADD NAMES DICTS
    ADD OPTION FOR BOX_WH/PT_DEFAULT

In [8]:
random.seed(0)

%run -m utils.inference_herdnet {imgs_dir} {model_weights} --size 640 --over 128

from pathlib import Path

dets_file = next(Path(imgs_dir).rglob("*_HerdNet_detections.csv"), None)

Building the model ...
Starting inference ...


  0%|          | 0/173 [00:00<?, ?it/s]

100%|██████████| 173/173 [05:22<00:00,  1.86s/it]

Inference finished!


### Read HerdNet output and run evaluation pipeline (no input required)

In [11]:
read_detections_HN(dets_file=dets_file, cls_name2id=class_name2ID, imgs_dir=imgs_dir, dor_thresh=dor_thresh, radii=radii, class_ids=list(radii.keys()), output_dir=output_dir,
                   ann_file=f"{imgs_dir}/test_annotations.json", ann_format="PT_DEFAULT")

class_ID2name = {cid: name for name,cid in class_name2ID.items()}
compute_errors_img_lvl(gt_counts_dir=f"{imgs_dir}/image_counts", pred_counts_dir=f"{output_dir}/detections", class_ids=list(class_ID2name.keys()), 
                           output_dir=output_dir)
compute_em_img_lvl(preds_dir=f"{output_dir}/detections", class_id2name=class_ID2name, task="locate", output_dir=output_dir)    

KeyError: 'Sheep/goat'